In [26]:
import numpy as np
from keras.utils import np_utils

PATH="/home/jango/distillation"

def tensor3(X,Y,timestep):
    Y=np_utils.to_categorical(Y,120)
    N_input=X.shape[1]
    N_output=Y.shape[1]
    tensorX=np.zeros((len(X),timestep,N_input))
    tensorY=np.zeros((len(X),N_output))
    tensorM=np.zeros((len(X),timestep))
    #append  timestep-1 zero_vectors at the begining
    M=np.hstack((np.zeros((timestep-1,)),np.ones(len(X),)))
    X=np.vstack((np.zeros((timestep-1,N_input)),X))

    for i in range(0,len(tensorM)):
        tensorX[i]=X[i:i+timestep]
        tensorY[i]=Y[i]
        tensorM[i]=M[i:i+timestep]
    return tensorX.astype("float32"),tensorY.astype("int16"),tensorM.astype("int16")

def AIO(X,Y,timestep):
    XX=list()
    YY=list()
    MM=list()
    for x,y in zip(X,Y):
        xx,yy,mm=tensor3(x,y,timestep)
        XX.append(xx)
        YY.append(yy)
        MM.append(mm)
    return np.concatenate(XX).astype("float32"),np.concatenate(YY).astype("int16"),np.concatenate(MM).astype("int16")

def LSTMmaker(MODEL,timestep=20):

#--------------------------------------------------------------------------
    source=PATH+"/%s/LSTMFile/" %MODEL

    X_train=np.load(source+MODEL[:3]+"_train_lstm.npy")
    y_train=np.load(source+MODEL[:3]+"_train_target_lstm.npy")

    X_train,Y_train,M_train=AIO(X_train,y_train,timestep)


    np.save(source+"%stimestep/%s_X_train_lstm.npy" %(timestep,MODEL[:3]),X_train)
    np.save(source+"%stimestep/%s_Y_train_lstm.npy" %(timestep,MODEL[:3]),Y_train)
    np.save(source+"%stimestep/%s_M_train_lstm.npy" %(timestep,MODEL[:3]),M_train)
LSTMmaker('teacher')
LSTMmaker('student')

In [2]:
import theano
import theano.tensor as T
import lasagne
import numpy as np
from keras.utils import np_utils
#****************************************************************************************
MODEL='teacher'
PATH="/home/jango/distillation"
X_VALI=np.load(PATH+"/"+MODEL+"/LSTMFile/%s_validation_lstm.npy" %MODEL[:3])

In [4]:
X_VALI[0].shape

(1090, 87)